## Cargar y parsear archivo

Usaremos la biblioteca `Biopython` que es muy útil para trabajar con secuencias biológicas.

In [1]:
%pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.0 MB/s eta 0:00:00


In [2]:
from Bio import SeqIO
import pandas as pd

In [3]:
fasta_pos = '/content/antifungal peptide_20180115.fasta'

## Secuencias positivas

In [4]:
data = []
try:
    for record in SeqIO.parse(fasta_pos, "fasta"):
        data.append({
            "ID": record.id,
            "Description": record.description,
            "Sequence": str(record.seq)
        })

    df_pos = pd.DataFrame(data)
    display(df_pos.head())

except FileNotFoundError:
    print(f"Error: El archivo no se encuentra en la ruta especificada: {fasta_pos}")
except Exception as e:
    print(f"Ocurrió un error al parsear el archivo fasta: {e}")

,ID,Description,Sequence
0,AP00001,AP00001,GLWSKIKEVGKEAAKAAAKAAGKAALGAVSEAV
1,AP00003,AP00003,DGVKLCDVPSGTWSGHCGSSSKCSQQCKDREHFAYGGACHYQFPSV...
2,AP00004,AP00004,NLCERASLTWTGNCGNTGHCDTQCRNWESAKHGACHKRGNWKCFCYFDC
3,AP00012,AP00012,GLFDIIKKIAESI
4,AP00013,AP00013,GLFDIIKKIAESF


In [8]:
# Seleccionar las secuencias positivas
df = df_pos[['Sequence']]

# Agregar una columna de label (si tiene actividad antifúngica o no)
df['label'] = 1

/tmp/ipython-input-4057750613.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = 1


## Concatenar ambas secuencias

In [9]:
df.head()

,Sequence,label
0,GLWSKIKEVGKEAAKAAAKAAGKAALGAVSEAV,1
1,DGVKLCDVPSGTWSGHCGSSSKCSQQCKDREHFAYGGACHYQFPSV...,1
2,NLCERASLTWTGNCGNTGHCDTQCRNWESAKHGACHKRGNWKCFCYFDC,1
3,GLFDIIKKIAESI,1
4,GLFDIIKKIAESF,1


In [10]:
df.shape

(1047, 2)

## Verificar duplicados

In [14]:
df["Sequence"].unique().shape

(1036,)

In [15]:
df = df.drop_duplicates().copy()

In [16]:
df.shape

(1036, 2)

## Verificar canónicos

In [17]:
# Definir aminoácidos canónicos
aminoacidos = set("ACDEFGHIKLMNPQRSTVWY")

In [18]:
# Verificar cuáles secuencias válidas
df["is_valid"] = [set(seq).issubset(aminoacidos) for seq in df["Sequence"]]

In [19]:
no_validas = df[~df["is_valid"]]
print("Secuencias inválidas:", len(no_validas))

Secuencias inválidas: 1


In [20]:
df_validas = df[df["is_valid"]].drop(columns="is_valid")
print("Secuencias válidas restantes:", len(df_validas))

Secuencias válidas restantes: 1035


In [21]:
display(df_validas)

,Sequence,label
0,GLWSKIKEVGKEAAKAAAKAAGKAALGAVSEAV,1
1,DGVKLCDVPSGTWSGHCGSSSKCSQQCKDREHFAYGGACHYQFPSV...,1
2,NLCERASLTWTGNCGNTGHCDTQCRNWESAKHGACHKRGNWKCFCYFDC,1
3,GLFDIIKKIAESI,1
4,GLFDIIKKIAESF,1
...,...,...
1042,FLSLIPKIATGIAALAKHL,1
1043,GIGSAILSAGKSIIKGLAKGLAEHF,1
1044,IIGPVLGLVGKALGGLL,1
1045,GLTRLFSVIK,1


In [22]:
df_validas = df_validas.rename(columns={'Sequence': 'sequence'})
display(df_validas.head())

,sequence,label
0,GLWSKIKEVGKEAAKAAAKAAGKAALGAVSEAV,1
1,DGVKLCDVPSGTWSGHCGSSSKCSQQCKDREHFAYGGACHYQFPSV...,1
2,NLCERASLTWTGNCGNTGHCDTQCRNWESAKHGACHKRGNWKCFCYFDC,1
3,GLFDIIKKIAESI,1
4,GLFDIIKKIAESF,1


In [23]:
df_validas.to_csv('MAMP-Pred_labeled.csv', index=False, header=True)

## Obtener metadatos

In [24]:
import json

In [25]:
def export_json(path_to_export, data_to_export):
    with open(path_to_export, 'w') as doc_export:
        json.dump(
            data_to_export,
            doc_export,
            indent=4,
            default=str,
            ensure_ascii=False)

In [26]:
def create_metada_with_multiple_values(df_metada_filter, full_df):
    dict_metadata = {}

    for column in df_metada_filter.columns:
        values = df_metada_filter[column].unique().tolist()

        if len(values)>1:
            values = [str(value) for value in values]
            values = ";".join(values)
            dict_metadata.update({column:values})
        else:
            dict_metadata.update({column:values[0]})

    dict_metadata.update({
        "number_of_sequences" : len(full_df)
    })

    return dict_metadata

In [27]:
def read_metadata(path_data, name_source):
    df_metada = pd.read_excel(path_data)
    df_metada_filter = df_metada[df_metada["name source"] == name_source]
    df_metada_filter = df_metada_filter[['type source',
                                         'estatico-dinamico',
                                         'licencia',
                                         'año de publicación',
                                         'fecha ultima actualizacion',
                                         'download date',
                                         'formato',
                                         'peptide property',
                                         'informacion del dataset',
                                         'unidad de medida',
                                         'Construccion de dataset negativos',
                                         'repositorio o servidor',
                                         'Publicacion',]]
    return df_metada_filter

In [28]:
df_metada_filter = read_metadata("/content/description_raw_data.xlsx", "MAMP-Pred")

In [29]:
dict_metadata = create_metada_with_multiple_values(df_metada_filter, df_validas)

In [30]:
export_json("metadata.json", dict_metadata)